In [21]:
import json
from pathlib import Path
import pandas as pd
import os

RESULTS_DIR = Path("evaluation_results")

In [22]:
# Aggregate results
results = []
for f in os.listdir(RESULTS_DIR):
    if f.endswith(".json"):
        with open(RESULTS_DIR / f, "r") as f:
            results.append(json.load(f))
        
# Save and print results
results_df = pd.DataFrame(results)

In [23]:
results_df['cot_correct'] = results_df['cot_answer'] == results_df['gold_answer']
results_df['finetuned_correct'] = results_df['finetuned_answer'] == results_df['gold_answer']
cot_accuracy = results_df['cot_correct'].mean()
finetuned_accuracy = results_df['finetuned_correct'].mean()
print(f"CoT accuracy: {cot_accuracy}")
print(f"Finetuned accuracy: {finetuned_accuracy}")

CoT accuracy: 0.17073170731707318
Finetuned accuracy: 0.08818011257035648


In [24]:
# see rows where critic was invoked
dfc = results_df[results_df['finetuned_output'].str.contains('<|endofturn|><|critic|>', regex=False)]
dfc

,question,gold_answer,cot_output,cot_answer,finetuned_output,finetuned_answer,cot_correct,finetuned_correct
1,An 8-year old child wants to buy a toy car whi...,4,To determine how many days it will take for th...,4,<|im_end|><|im_end|><|im_end|><|im_end|><|im_e...,None,True,False
2,"Elliott is trying to walk 10,000 steps a day. ...",2000,Let's break down the information given:\n\n1. ...,None,<|im_end|><|im_end|><|im_end|><|im_end|><|im_e...,None,False,False
4,Tom plants 10 trees a year. Every year he als...,91,Let's break this down step by step:\n\n1. Tom ...,None,<|im_end|><|im_end|><|im_end|><|im_end|><|im_e...,None,False,False
5,John wins an award at work. The award has a 1...,"114,200",To calculate John's total earnings for the yea...,None,<|im_end|><|im_end|><|im_end|><|im_end|><|im_e...,None,False,False
6,Terry eats 2 yogurts a day. They are currentl...,75,To determine how much Terry spends on yogurt o...,None,<|im_end|><|im_end|><|im_end|><|im_end|><|im_e...,None,False,False
...,...,...,...,...,...,...,...,...
527,Nissa hires 60 seasonal workers to play elves ...,30,Let's break down the problem step by step:\n\n...,30,<|im_end|><|im_end|><|im_end|><|im_end|><|im_e...,None,True,False
528,"When Freda cooks canned tomatoes into sauce, t...",12,To determine how many tomatoes Freda used to m...,24,<|im_end|><|im_end|><|im_end|><|im_end|><|im_e...,None,False,False
529,Carmen goes to an auction to win an antique de...,500,Let's break down the problem step-by-step:\n\n...,None,<|im_end|><|im_end|><|im_end|><|im_end|><|im_e...,None,False,False
530,Mark is trying to choose between two venues fo...,10,To determine how many guests are necessary for...,None,<|im_start|>system\nYou are a helpful assistan...,None,False,False


In [19]:
s = dfc['finetuned_output'].iloc[12]
segs = s[s.find('<|solver|>', s.find('<|solver|>') + 1):].split('<|endofturn|>')[:-1]
print(s.strip("<|im_end|>"))

start|>system
You are a helpful assistant. Final answer format: {"answer": <number>}.<|im_end|>
<|im_start|>user
<|question|> A shoe store was having a weekend sale on a brand of popular tennis shoes. On Friday the store sold 14 pairs of tennis shoes. The next day they sold double that number of shoes. On the last day of the sale they sold one-half the amount that they did the day before, but six people returned their pairs because they didn't fit. How many tennis shoes were sold by the end of the sale?<|im_end|>
<|im_start|>assistant
<|solver|> Let's break down the information given and calculate step by step:
- On Friday, the store sold 14 pairs of tennis shoes.
- On Saturday (double the previous day), they sold 2 * 14 = 28 pairs.
- On Sunday (half of what was sold on Saturday), they sold 0.5 * 28 = 14 pairs. However, 6 people returned their pairs due to not fitting, so we need to subtract those from the total sales on Sunday: 14 - 6 = 8 pairs sold on Sunday.

Now let's add up all th